# 2.0 Feature Engineering

## 2.1 Angle and Distance Features

The `PoseDimensionCalculator` class operates through a series of algorithmic steps to calculate dimensions and angles based on pose data. Here's a technical breakdown of its key functions and the formulas used:

1. `enhance_pose_landmarks()`:  This method refines the pose data by creating additional landmark points to represent key body parts more accurately. It approximates the spine's location using the average coordinates of the torso's four landmarks (head, chest, stomach, and hips) and simplifies hand landmarks by averaging the coordinates of the pinky and index fingers to estimate the central position of the knuckles.
2. `calculate_pose_distance()` & `calculate_distances()`: This method calculates the physical distances between two joints. A smaller distance means a closer contact. elbow to knuckles.
3. `calculate_pose_angle()` & `calculate_connected_joint_range()`: These methods use `arctan2` to compute the angle between two vectors (connected joints based on human anatomy).
4. `calculate_rate_of_change()`: The method calculates the per-second rate of change of distances.

Concepts:
* Eucleandian Distance: https://science.howstuffworks.com/math-concepts/distance-formula.htm
* Atan2: https://en.wikipedia.org/wiki/Atan2
* Velocity: https://en.wikipedia.org/wiki/Velocity

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import mode

class PoseDimensionCalculator:
    
    def __init__(self, data, is_video=False):
        self.data = data
        
        self.enhance_pose_landmarks = self.enhance_pose_landmarks()
        self.data = self.enhance_pose_landmarks

        distances = self.calculate_distances()
        self.data = pd.concat([self.data, distances], axis=1)
        
        angles = self.calculate_connected_joint_range()
        self.data = pd.concat([self.data, angles], axis=1)

        if is_video:
            merged_data = self.calculate_rate_of_change()
            self.data = merged_data

    def enhance_pose_landmarks(self):
        enhanced_data = []
        for index, row in self.data.iterrows():
            enhanced_row = self.process_row(row)
            enhanced_row['index'] = index
            enhanced_data.append(enhanced_row)
        enhanced_df = pd.DataFrame(enhanced_data).set_index('index')  # Set the index of the DataFrame
        merged = pd.concat([self.data, enhanced_df], axis=1)  # Merge with the original DataFrame
        return merged

    def process_row(self, row):
        x_cols = sorted([col for col in self.data.columns if col.endswith('_x')])
        y_cols = sorted([col for col in self.data.columns if col.endswith('_y')])
        z_cols = sorted([col for col in self.data.columns if col.endswith('_z')])        
        x = row[x_cols].values
        y = row[y_cols].values
        z = row[z_cols].values

        enhanced_row = {
            'chest_x': x[11:13].mean(),
            'chest_y': y[11:13].mean(), 
            'chest_z': z[11:13].mean(),
            'stomach_x': (x[11:13].mean() + x[23:25].mean()) / 2,
            'stomach_y': (y[11:13].mean() + y[23:25].mean()) / 2,
            'stomach_z': (z[11:13].mean() + z[23:25].mean()) / 2,
            'hip_x': x[23:25].mean(),
            'hip_y': y[23:25].mean(),
            'hip_z': z[23:25].mean(),
        }

        return enhanced_row

    def calculate_pose_distance(self, Ax, Ay, Bx, By):
        distance = np.sqrt((self.data[Ax] - self.data[Bx])**2 + (self.data[Ay] - self.data[By])**2)
        return np.round(distance, 4)

    def calculate_distances(self):
        dist = pd.DataFrame(index=self.data.index)
        dist['d_nose_to_rgt_knee'] = self.calculate_pose_distance('landmark_00_x', 'landmark_00_y', 'landmark_25_x', 'landmark_25_y')
        dist['d_nose_to_lft_knee'] = self.calculate_pose_distance('landmark_00_x', 'landmark_00_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_rgt_shoulder_to_ankle'] = self.calculate_pose_distance('landmark_11_x', 'landmark_11_y', 'landmark_25_x', 'landmark_25_y')
        dist['d_lft_shoulder_to_ankle'] = self.calculate_pose_distance('landmark_12_x', 'landmark_12_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_wrists'] = self.calculate_pose_distance('landmark_15_x', 'landmark_15_y', 'landmark_16_x', 'landmark_15_y')
        dist['d_elbows'] = self.calculate_pose_distance('landmark_13_x', 'landmark_13_y', 'landmark_14_x', 'landmark_14_y')
        dist['d_knees'] = self.calculate_pose_distance('landmark_25_x', 'landmark_25_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_ankles'] = self.calculate_pose_distance('landmark_27_x', 'landmark_27_y', 'landmark_28_x', 'landmark_28_y')
        return dist

    def calculate_pose_angle(self, Ax, Ay, Bx, By, Cx, Cy):
        A = self.data[[Ax, Ay]].values
        B = self.data[[Bx, By]].values
        C = self.data[[Cx, Cy]].values
        BA = A - B
        BC = C - B
        angle_BA = np.arctan2(BA[:, 1], BA[:, 0])
        angle_BC = np.arctan2(BC[:, 1], BC[:, 0])
        angle_difference = np.degrees(angle_BC - angle_BA)    
        return np.round(angle_difference, 4)
        
    def calculate_connected_joint_range(self):
        
        angles = pd.DataFrame(index=self.data.index)
        angles['a_nose_to_rgt_shoulder'] = self.calculate_pose_angle('landmark_00_x', 'landmark_00_y', 'chest_x', 'chest_y','landmark_12_x','landmark_12_y')
        angles['a_nose_to_lft_shoulder'] = self.calculate_pose_angle('landmark_00_x', 'landmark_00_y', 'chest_x', 'chest_y','landmark_11_x','landmark_12_y')
        angles['a_mid_hip_to_knees'] = self.calculate_pose_angle('landmark_26_x','landmark_26_y','hip_x', 'hip_y','landmark_25_x','landmark_25_y')
        angles['a_lft_shoulder_to_wrist'] = self.calculate_pose_angle('landmark_12_x', 'landmark_12_y', 'landmark_14_x', 'landmark_14_y', 'landmark_16_x', 'landmark_16_y')
        angles['a_lft_hip_to_ankle'] = self.calculate_pose_angle('landmark_24_x', 'landmark_24_y', 'landmark_26_x', 'landmark_26_y','landmark_28_x', 'landmark_28_y')
        angles['a_rgt_shoulder_to_wrist'] = self.calculate_pose_angle('landmark_11_x', 'landmark_11_y', 'landmark_13_x', 'landmark_13_y', 'landmark_15_x', 'landmark_15_y')
        angles['a_rgt_hip_to_ankle'] = self.calculate_pose_angle('landmark_23_x', 'landmark_23_y', 'landmark_25_x', 'landmark_25_y','landmark_27_x', 'landmark_27_y')
        
        return angles
        
    def calculate_rate_of_change(self):
        secs_counts = self.data.groupby('secs').size()
        fps = secs_counts.max()

        d_cols = sorted([col for col in self.data.columns if col.startswith('dist_')])

        grouped_data = self.data.groupby('secs')[d_cols].mean()
        roc_data = pd.DataFrame(index=grouped_data.index)
        
        for col in d_cols:
            differences = grouped_data[col].diff().fillna(0)
            roc_data[f'r_{col}'] = differences * fps
            
        merged_data = pd.merge(self.data, roc_data, on='secs', how='left')
        return merged_data

In [7]:
data = pd.read_csv("dataset/pose_data_augmented.csv")
results = PoseDimensionCalculator(data, is_video=False)
results.data.to_csv("dataset/pose_data_augmented_res.csv", index=False)
results.data

,image_filename,pose_name,label,label_encoded,theta,landmark_00_x,landmark_00_y,landmark_00_z,landmark_01_x,landmark_01_y,...,d_elbows,d_knees,d_ankles,a_nose_to_rgt_shoulder,a_nose_to_lft_shoulder,a_mid_hip_to_knees,a_lft_shoulder_to_wrist,a_lft_hip_to_ankle,a_rgt_shoulder_to_wrist,a_rgt_hip_to_ankle
0,advanced-figurehead-lft.png,advanced-figurehead-lft,advanced-figurehead,0,0,0.545408,0.740922,-0.407526,0.529543,0.728610,...,0.0334,0.0267,0.0124,10.7029,-122.8088,0.6969,291.3413,98.4931,276.8157,102.7408
1,advanced-figurehead-lft.png,advanced-figurehead-lft,advanced-figurehead,0,10,0.607888,0.740922,-0.306625,0.586982,0.728610,...,0.1425,0.1129,0.1207,-137.2727,33.0201,-25.9627,294.2652,94.5808,268.7887,107.6087
2,advanced-figurehead-lft.png,advanced-figurehead-lft,advanced-figurehead,0,20,0.651898,0.740922,-0.196408,0.626586,0.728610,...,0.2668,0.2357,0.2498,-134.0704,41.4422,-51.8355,295.3539,88.8913,254.5938,110.7964
3,advanced-figurehead-lft.png,advanced-figurehead-lft,advanced-figurehead,0,30,0.676100,0.740922,-0.080224,0.647152,0.728610,...,0.3841,0.3521,0.3714,-126.3352,50.6887,-74.8649,294.9057,81.4701,231.1617,112.4588
4,advanced-figurehead-lft.png,advanced-figurehead-lft,advanced-figurehead,0,40,0.679759,0.740922,0.038399,0.648054,0.728610,...,0.4901,0.4580,0.4817,-114.6003,63.1216,-92.7334,293.0172,72.4473,200.5086,112.6505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19579,yogini-twisted-rgt.png,yogini-twisted-rgt,yogini-twisted,271,310,0.221597,0.731075,-0.655868,0.241815,0.751349,...,0.5911,0.5505,0.6199,121.7164,-47.0931,154.3500,172.0633,31.9434,-228.7234,-201.5062
19580,yogini-twisted-rgt.png,yogini-twisted-rgt,yogini-twisted,271,320,0.332121,0.731075,-0.607424,0.348152,0.751349,...,0.5291,0.5494,0.6267,140.0934,-26.8885,154.1885,174.1254,36.1317,-214.9999,-209.2061
19581,yogini-twisted-rgt.png,yogini-twisted-rgt,yogini-twisted,271,330,0.432553,0.731075,-0.540524,0.443911,0.751349,...,0.4541,0.5319,0.6252,147.7341,-16.1314,153.2878,175.4786,41.6074,-199.8780,-216.7328
19582,yogini-twisted-rgt.png,yogini-twisted-rgt,yogini-twisted,271,340,0.519843,0.731075,-0.457200,0.526182,0.751349,...,0.3702,0.4986,0.6154,149.7391,-8.2069,151.5254,176.4539,48.8414,-183.9328,-224.2409
